In [1]:
import importlib
import numpy as np
import pandas as pd
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [3]:
from keras.models import Model, load_model
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, Flatten
from keras.models import Model

Using TensorFlow backend.


In [4]:
emb = utils.load_embedding('../embedding.csv')

In [5]:
emb.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.060974,-0.317879,-0.550560,-0.202091,0.760389,-1.226085,0.875159,0.459753,-1.196949,1.053209,...,-0.597071,-0.066059,0.345023,-0.429354,0.384491,-0.397192,-0.155742,0.372814,0.742573,0.722736
1,-0.095839,-0.077554,-0.355692,1.049146,-0.590769,0.929406,-0.162880,-0.456715,-0.988432,-0.104014,...,-1.097297,-0.756459,-0.336356,0.792779,-0.133896,-0.490780,0.375516,-0.164238,1.186742,0.085916
2,-0.584020,0.390310,0.652820,-0.340300,0.194930,-0.834890,0.119290,-0.572910,-0.568440,0.729890,...,0.285860,-0.052162,-0.508180,-0.634590,0.338890,0.284160,-0.203400,-1.233800,0.467150,0.788580
3,0.257690,0.456290,-0.769740,-0.376790,0.592720,-0.063527,0.205450,-0.573850,-0.290090,-0.136620,...,0.030498,-0.395430,-0.385150,-1.000200,0.087599,-0.310090,-0.346770,-0.314380,0.750040,0.970650
4,-0.695426,0.591739,-0.397985,1.042686,0.069218,1.111737,-0.025059,0.280132,0.155576,-0.393120,...,0.651884,-0.799066,0.226521,0.059271,0.521701,-0.369458,0.217679,-0.987510,-0.522543,0.546550


In [6]:
emb_dim = emb.shape[1]

In [7]:
emb_dim

50

In [9]:
word2ind, ind2word = utils.load_index_word_map('../word2ind', '../ind2word')

In [10]:
vocab_size = len(word2ind)

In [7]:
dataset = utils.load_training_samples()

In [8]:
train_x, val_x, train_y, val_y = utils.train_valid_split(dataset.values[:, :-1], dataset.values[:, -1:], test_size=0.3, random_state=43)

In [9]:
train_x.shape

(7961414, 100)

In [10]:
train_y.shape

(7961414, 1)

In [11]:
val_x.shape

(3412034, 100)

In [12]:
val_y.shape

(3412034, 1)

In [13]:
input_length = train_x.shape[1]

In [14]:
model = load_model('model20180127_3.h5')

In [15]:
emb = model.get_weights()[0]

In [16]:
emb_dim = emb.shape[1]

In [17]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
embedding = Embedding(vocab_size, emb_dim)
encoder_embedding_outputs = embedding(encoder_inputs)
encoder_lstm_dim = 100
_, state_h, state_c = LSTM(encoder_lstm_dim, return_state=True)(encoder_embedding_outputs)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
decoder_embedding_outputs = embedding(decoder_inputs)
decoder_lstm_dim = 100
decoder_lstm_outputs = LSTM(decoder_lstm_dim, return_sequences=True)(decoder_embedding_outputs, initial_state=encoder_states)
decoder_outputs = Dense(vocab_size, activation='softmax')(decoder_lstm_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
# Compile & run training
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=2,
          epochs=epochs,
          validation_split=0.2)

In [15]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, None)          0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, None)          0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, None, 50)      4072600     input_3[0][0]                    
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, None, 50)      4072600     input_4[0][0]                    
___________________________________________________________________________________________

In [19]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = ind2word[str(sampled_token_index)]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence